In [1]:
import os

In [2]:
%pwd

'c:\\MLOps\\Chest-Disease\\Chest_Disease_Detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\MLOps\\Chest-Disease\\Chest_Disease_Detection'

In [12]:
ls

 Volume in drive C is Acer
 Volume Serial Number is BC76-A51F

 Directory of c:\MLOps\Chest-Disease\Chest_Disease_Detection

11-05-2025  11:25    <DIR>          .
10-05-2025  15:00    <DIR>          ..
11-05-2025  11:13             3,630 .gitignore
11-05-2025  11:13    <DIR>          artifacts
10-05-2025  15:01    <DIR>          config
10-05-2025  15:01                 0 dvc.yaml
10-05-2025  15:00             1,089 LICENSE
11-05-2025  10:58    <DIR>          logs
11-05-2025  11:25               434 main.py
11-05-2025  11:12                12 params.yaml
11-05-2025  10:48               961 README.md
10-05-2025  15:06               194 requirements.txt
11-05-2025  11:14    <DIR>          research
10-05-2025  15:08               806 setup.py
11-05-2025  11:25    <DIR>          src
10-05-2025  15:01    <DIR>          templates
10-05-2025  15:01             1,280 templates.py
               9 File(s)          8,406 bytes
               8 Dir(s)  232,661,622,784 bytes free


In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
ls

 Volume in drive C is Acer
 Volume Serial Number is BC76-A51F

 Directory of c:\MLOps\Chest-Disease\Chest_Disease_Detection

11-05-2025  10:58    <DIR>          .
10-05-2025  15:00    <DIR>          ..
10-05-2025  15:00             3,617 .gitignore
10-05-2025  15:01    <DIR>          config
10-05-2025  15:01                 0 dvc.yaml
10-05-2025  15:00             1,089 LICENSE
11-05-2025  10:58    <DIR>          logs
11-05-2025  11:12                12 params.yaml
11-05-2025  10:48               961 README.md
10-05-2025  15:06               194 requirements.txt
11-05-2025  11:04    <DIR>          research
10-05-2025  15:08               806 setup.py
10-05-2025  15:20    <DIR>          src
10-05-2025  15:01    <DIR>          templates
10-05-2025  15:01             1,280 templates.py
               8 File(s)          7,959 bytes
               7 Dir(s)  232,690,196,480 bytes free


In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-11 11:13:15,138: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-11 11:13:15,142: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-11 11:13:15,147: INFO: common: created directory at: artifacts]
[2025-05-11 11:13:15,150: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-11 11:13:15,153: INFO: 1983665874: Downloading data from https://drive.google.com/file/d/19dssf7Msyw7FDQm-9O3B1H7Av6v0WDry/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=19dssf7Msyw7FDQm-9O3B1H7Av6v0WDry
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=19dssf7Msyw7FDQm-9O3B1H7Av6v0WDry&confirm=t&uuid=fce22989-411e-4a5a-b454-fcfe0d970680
To: c:\MLOps\Chest-Disease\Chest_Disease_Detection\artifacts\data_ingestion\data.zip
100%|██████████| 124M/124M [01:47<00:00, 1.15MB/s] 

[2025-05-11 11:15:08,074: INFO: 1983665874: Downloaded data from https://drive.google.com/file/d/19dssf7Msyw7FDQm-9O3B1H7Av6v0WDry/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
